In [ ]:
# セル 1: 必要なライブラリのインポート
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import torch
import numpy as np
from pathlib import Path
import threading
import queue
import time
import os
from loguru import logger
import io

# fish-speech関連のモジュールをインポート
# (fish-speechがインストールされ、パスが通っている必要があります)
# 必要に応じて sys.path に fish-speech のパスを追加してください
# import sys
# sys.path.append('/path/to/fish-speech') # 例
try:
    from fish_speech.inference_engine import TTSInferenceEngine
    from fish_speech.models.text2semantic.inference import launch_thread_safe_queue
    from fish_speech.models.vqgan.inference import load_model as load_decoder_model
    from fish_speech.utils.schema import ServeTTSRequest
except ImportError as e:
    print(f"Error importing fish-speech modules: {e}")
    print("Please ensure fish-speech is installed and accessible in your Python path.")
    # ここで処理を中断するか、ダミーの関数を定義するなどの対応が必要です
    raise e

print("必要なライブラリをインポートしました。")

In [ ]:
# セル 2: 設定値 (モデルパスなどを環境に合わせて変更してください)

# --- ユーザー設定 ---
LLAMA_CHECKPOINT_PATH = Path("./checkpoints/fish-speech-1.5") # LLAMAモデルのパス
DECODER_CHECKPOINT_PATH = Path("./checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth") # VQGANモデルのパス
DECODER_CONFIG_NAME = "firefly_gan_vq" # VQGANモデルの設定名
# --------------------

# デバイス設定 (自動検出)
if torch.cuda.is_available():
    DEVICE = "cuda"
    PRECISION = torch.bfloat16 # または torch.float16
elif torch.backends.mps.is_available():
    DEVICE = "mps"
    PRECISION = torch.float16 # MPSはbfloat16をサポートしない場合がある
else:
    DEVICE = "cpu"
    PRECISION = torch.float32

COMPILE_MODEL = True # モデルコンパイルを有効にするか (高速化のため推奨)

# WindowsかつCompile=Trueの場合、警告を出す (動作しない可能性があるため)
if os.name == 'nt' and COMPILE_MODEL:
    print("警告: Windows環境でのモデルコンパイルは不安定な場合があります。エラーが発生する場合は COMPILE_MODEL = False を試してください。")

# Make einx happy (fish-speechが必要とする可能性あり)
os.environ["EINX_FILTER_TRACEBACK"] = "false"

print(f"デバイス: {DEVICE}")
print(f"精度: {PRECISION}")
print(f"モデルコンパイル: {COMPILE_MODEL}")
print(f"LLAMAモデルパス: {LLAMA_CHECKPOINT_PATH}")
print(f"VQGANモデルパス: {DECODER_CHECKPOINT_PATH}")

In [ ]:
# セル 3: モデルの読み込みとコンパイル (このセルは時間がかかります)

print("モデルの読み込みとコンパイルを開始します。時間がかかる場合があります...")
start_time = time.time()

# 1. LLAMAモデルを別スレッドで読み込み&コンパイル
try:
    print("LLAMAモデルを読み込んでいます...")
    llama_queue = launch_thread_safe_queue(
        checkpoint_path=LLAMA_CHECKPOINT_PATH,
        device=DEVICE,
        precision=PRECISION,
        compile=COMPILE_MODEL,
    )
    print("LLAMAモデルの読み込み完了。")
except Exception as e:
    print(f"LLAMAモデルの読み込み中にエラーが発生しました: {e}")
    if COMPILE_MODEL:
        print("コンパイルなしで再試行します (COMPILE_MODEL = False)...")
        COMPILE_MODEL = False
        llama_queue = launch_thread_safe_queue(
            checkpoint_path=LLAMA_CHECKPOINT_PATH,
            device=DEVICE,
            precision=PRECISION,
            compile=COMPILE_MODEL,
        )
        print("LLAMAモデルの読み込み完了 (コンパイルなし)。")
    else:
        raise e

# 2. VQGANデコーダーモデルの読み込み
print("VQGANデコーダーモデルを読み込んでいます...")
decoder_model = load_decoder_model(
    config_name=DECODER_CONFIG_NAME,
    checkpoint_path=DECODER_CHECKPOINT_PATH,
    device=DEVICE,
)
print("VQGANデコーダーモデルの読み込み完了。")

# 3. TTS推論エンジンの初期化
print("TTS推論エンジンを初期化しています...")
inference_engine = TTSInferenceEngine(
    llama_queue=llama_queue,
    decoder_model=decoder_model,
    compile=COMPILE_MODEL, # コンパイル設定をエンジンにも渡す
    precision=PRECISION,
)
print("TTS推論エンジンの初期化完了。")

# 4. ウォームアップ実行 (初回推論の遅延を減らすため)
print("モデルのウォームアップを実行中...")
try:
    _ = list(inference_engine.inference(ServeTTSRequest(text="warm up")))
    print("ウォームアップ完了。")
except Exception as e:
    print(f"ウォームアップ中にエラーが発生しました: {e}")
    print("アプリは起動しますが、初回の音声生成に時間がかかる可能性があります。")

end_time = time.time()
print(f"モデルの準備完了。所要時間: {end_time - start_time:.2f} 秒")

# グローバル変数としてエンジンを保持
tts_engine_global = inference_engine

In [ ]:
# セル 4: ipywidgets UIの定義と表示 (修正版)

# --- UI要素の定義 ---
app_title = widgets.HTML("<h1>GMC Speech</h1>")
text_input = widgets.Textarea(
    # placeholderを修正
    placeholder='ここにテキストを入力するか、下のボタンからファイルをアップロードして内容を編集してください。',
    layout=widgets.Layout(width='95%', height='150px')
)
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    # descriptionを修正
    description='テキスト読込(&編集)',
    tooltip='テキストファイルを読み込み、テキストエリアに表示して編集できます'
)
generate_button = widgets.Button(
    description='音声生成',
    button_style='success',
    tooltip='入力/編集されたテキストから音声を生成します',
    icon='play',
    layout=widgets.Layout(width='auto')
)
status_label = widgets.Label(value="")
audio_output = widgets.Output()

# --- UIレイアウト ---
ui_layout = widgets.VBox([
    app_title,
    text_input,
    widgets.HBox([file_upload, generate_button]),
    status_label,
    audio_output
])

# --- UI表示 ---
display(ui_layout)

print("UIを表示しました。") # 完了メッセージを追加

In [ ]:
# セル 5: イベントハンドラの定義 (修正版)

# ファイルアップロード時の処理
def on_file_upload_change(change):
    # change['new'] はアップロードされたファイルのタプル (通常は1要素)
    # change['old'] は前の値 (通常は空のタプル)
    uploaded_files = change['new']
    if uploaded_files:
        # 最初のファイルを取得 (multiple=False なので常に1つのはず)
        uploaded_file_info = uploaded_files[0]
        try:
            # ファイル内容をデコードしてテキストエリアに設定
            content = uploaded_file_info['content'].tobytes().decode('utf-8') # contentはmemoryviewの場合があるのでtobytes()
            text_input.value = content # ★ 読み込んだ内容をTextareaに設定
            status_label.value = f"ファイル '{uploaded_file_info['name']}' を読み込みました。内容は編集可能です。"
        except Exception as e:
            status_label.value = f"ファイルの読み込み/デコードに失敗しました: {e}"
            logger.error(f"File upload error: {e}")
        finally:
            # ★ ファイルアップロードウィジェットの値をリセットして、同じファイルを再度アップロードできるようにする
            #    ipywidgetsのバージョンによって挙動が異なる可能性があるため、複数の方法を試す
            try:
                file_upload.value = () # タプルでリセット
            except Exception:
                 try:
                     file_upload.value = [] # リストでリセット
                 except Exception as reset_e:
                     logger.error(f"Failed to reset FileUpload widget: {reset_e}")
                     # どうしてもリセットできない場合は、ユーザーに手動でのクリアを促すメッセージなどを表示
                     # status_label.value += " (アップロードウィジェットのリセットに失敗しました)"

    # change['old'] を使ってファイル選択がキャンセルされたことを検知することも可能だが、
    # ここでは new が空でない場合のみ処理するようにしているため、キャンセル時の特別な処理は不要。

# 音声生成ボタンクリック時の処理 (変更なし)
def on_generate_button_clicked(b):
    # 既存の音声出力をクリア
    with audio_output:
        clear_output(wait=True)

    input_text = text_input.value.strip()
    if not input_text:
        status_label.value = "テキストが入力されていません。"
        return

    # ボタンを無効化し、ステータスを更新
    generate_button.disabled = True
    status_label.value = "音声生成中... (テキストの長さによっては時間がかかります)"

    try:
        start_gen_time = time.time()

        # TTS推論リクエストを作成
        req = ServeTTSRequest(
            text=input_text,
            # 必要に応じて他のパラメータ (chunk_length, top_pなど) を設定
        )

        # グローバル変数から推論エンジンを取得
        engine = tts_engine_global

        # 音声生成を実行
        generated_audio_data = None
        sample_rate = 44100 # デフォルト値

        for result in engine.inference(req):
            if result.code == "final" and result.audio is not None:
                sample_rate, generated_audio_data = result.audio
                break
            elif result.code == "error":
                raise result.error

        end_gen_time = time.time()

        if generated_audio_data is not None:
            status_label.value = f"音声生成完了！ ({end_gen_time - start_gen_time:.2f} 秒)"
            with audio_output:
                display(Audio(data=generated_audio_data, rate=sample_rate))
        else:
            status_label.value = "音声の生成に失敗しました（データがありません）。"

    except Exception as e:
        status_label.value = f"エラーが発生しました: {e}"
        logger.error(f"Inference error: {e}")
        import traceback
        logger.error(traceback.format_exc())

    finally:
        # ボタンを再度有効化
        generate_button.disabled = False

print("イベントハンドラを定義しました。") # 完了メッセージを追加

In [ ]:
# セル 6: イベントハンドラとウィジェットの連携設定

# FileUploadウィジェットの値が変更されたらon_file_upload_changeを呼び出す
file_upload.observe(on_file_upload_change, names='value')

# Buttonウィジェットがクリックされたらon_generate_button_clickedを呼び出す
generate_button.on_click(on_generate_button_clicked)

print("イベントハンドラを設定しました。アプリを使用できます。")